<a href="https://colab.research.google.com/github/gawankarsanket/classification_networkintrusion/blob/main/Capstone_Classification_NetSecML_Inference_BaseModel_via_FastAPI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introduction:
We already have developed a basemodel for Network Intrusion Detection. https://colab.research.google.com/drive/1QaCaBydr4DktlOwPIdOWjU8cAqB8OsQN This model is trained with XGB classifier with accuracy of 99% and ready for inference.

We have saved this model in S3 bucket. Our objective is to download this model from S3 and then use it to inference

To inference, we have build API, using FastAPI that will get user input file and then will display the prediction on screen.
Same time the output/prediction file will be pushed to S3 for records


In [ ]:
pip install fastapi uvicorn nest-asyncio pandas boto3 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 953.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

#Access S3 for to laod trained model

In [ ]:
#import python SDK ie API for AWS services
import boto3


#Access your secret keys in Python via:
from google.colab import userdata

BUCKET_NAME =userdata.get('bucketName')
ACCESS_KEY= userdata.get('accessKey')
SECRET_KEY= userdata.get('secretKey')


# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key= SECRET_KEY)

#s3 is object created which is  authrosied using access keys & secret key to  access the S3 bucket in AWS

In [ ]:
# save the filename in KEY variable
KEY = 'capstoneClassification.pkl' # replace with your object key

try:
  # we are trying to download file from s3 with name `capstoneClassification.pkl` stored in varaible KEY
  # to colab dir with name `capstoneClassification.pkl`
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'capstoneClassification.pkl')

except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

#FastAPI method

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pickle
import pandas as pd
import boto3
import nest_asyncio
import uvicorn

# Initialize FastAPI app
app = FastAPI()

# Load the model and preprocessing objects
with open('capstoneClassification.pkl', 'rb') as f:
    trained_model = pickle.load(f)
    le = pickle.load(f)
    feature_selected = pickle.load(f)

def encoding(df):
    for col in df.columns:
        if df[col].dtypes == 'object':
            name = 'num' + col  # creating heading for that column
            df[name] = le.fit_transform(df[col])  # Column with encoded label is added
            df.drop(col, axis=1, inplace=True)  # column with object type will be removed
    return df

def prediction_function(xtest):
    ypred = trained_model.predict(xtest)
    return ypred

@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    try:
        input_file = pd.read_csv(file.file)
        input_file.drop('class', axis=1, inplace=True)
        input_df = encoding(input_file.copy())
        input_df_selectedCols = input_df[feature_selected]
        ypred = prediction_function(input_df_selectedCols)

        input_file['IsMalicious'] = ypred
        output_mapping = {0: 'Normal', 1: 'Attack'}
        input_file['class'] = input_file['IsMalicious'].map(output_mapping)

        # Save the results to a CSV file
        input_file.to_csv('Prediction.csv', index=False)

        # Upload the file to S3
        push_to_s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
        push_to_s3.upload_file('Prediction.csv', BUCKET_NAME, 'Prediction.csv')

        # Return the predictions in the response
        return {"filename": file.filename, "status": "Upload successful", "predictions": input_file.to_dict(orient="records")}
    except Exception as e:
        return JSONResponse(status_code=500, content={"message": str(e)})

if __name__ == "__main__":
    # Required to run FastAPI app in Colab
    nest_asyncio.apply()

    # Start ngrok and get the public URL
    public_url = ngrok.connect(port=8000)
    print(f"Public URL: {public_url}")

    # Run FastAPI app with Uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


ERROR:pyngrok.process.ngrok:t=2024-05-21T19:59:37+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2024-05-21T19:59:37+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2024-05-21T19:59:37+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authto

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
from pyngrok import ngrok

In [ ]:
pip install pyngrok -q

In [ ]:
AUTHTOKEN=userdata.get('ngrockTunnelToken')

In [ ]:
!ngrok authtoken AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!cat /root/.config/ngrok/ngrok.yml

region: us
version: '2'
authtoken: AUTHTOKEN
